# Integrantes

- Bruschi, Tomas Iago: 21037/2
- Gonzalez, Ivan: 21816/3
- Percara, Francisco Gonzalo: 21024/7

# Motivacion de seleccion

### Justificación de la selección del dataset *Wine Quality*

El dataset *Wine Quality* fue seleccionado por presentar un conjunto de datos **numéricamente consistente, completo y adecuado para el análisis estadístico y la modelización matemática**, alineado con los objetivos de la materia **Matemática IV**.

#### 1. Naturaleza del dataset

El conjunto contiene exclusivamente **variables numéricas continuas**, lo que permite aplicar directamente herramientas estadísticas y matemáticas sin necesidad de codificación o tratamiento de variables categóricas. Entre las variables se incluyen medidas fisicoquímicas del vino como acidez, pH, densidad, alcohol, sulfatos y cloruros, además de una variable objetivo numérica (*quality*) que representa una evaluación sensorial promedio.
Esto facilita el uso de **métodos de regresión lineal, análisis multivariable, correlaciones y normalización**, todos temas centrales en la cursada.

#### 2. Ventajas técnicas

* **Escalabilidad y limpieza:** no presenta valores faltantes ni inconsistencias, lo que permite concentrarse en el análisis matemático sin tareas extensas de preprocesamiento.
* **Dimensionalidad apropiada:** cuenta con alrededor de 1600 observaciones y 11 variables predictoras, cantidad ideal para análisis numéricos en entorno Python.

#### 3. Pertinencia con la materia

El conjunto de datos posibilita explorar en profundidad conceptos vistos en Matemática III, tales como:

* **Correlación y covarianza entre variables.**
* **Modelos de regresión lineal múltiple y ajuste de parámetros.**
* **Normalización y estandarización de variables.**
* **Cálculo de métricas de error, R² y R² ajustado.**

# Regresión Lineal Simple

## a) Definir la variable respuesta y las variables predictoras, justificando el motivo de la elección de estas.

La variable respuesta será **Quality**, ya que el dataset busca medir la calidad de los vinos tintos en base a sus características principales.

Las variables predictoras serán: 
- **Volatile Acidity**: la acidez volátil (principalmente ácido acético) está asociada a defectos en el vino si es demasiado alta. Niveles elevados producen un sabor y aroma desagradables a vinagre, afectando negativamente la calidad. 
- **Density**: está estrechamente relacionada con el contenido de azúcar y alcohol. Los vinos con menor densidad suelen tener un mayor contenido alcohólico (ya que el azúcar se transformó en alcohol), lo que generalmente indica un proceso de fermentación completo y buena calidad. 
- **Sulphates**: actúan como conservantes y antioxidantes. Una cantidad equilibrada ayuda a mantener la frescura y estabilidad del vino, mientras que un exceso puede alterar su aroma y sabor. Los vinos de alta calidad tienden a tener un nivel adecuado de sulfitos. 
- **Alcohol**: es uno de los factores más directamente correlacionados con la calidad. Los vinos con mayor contenido alcohólico suelen percibirse como de mejor calidad debido a su cuerpo, aroma y persistencia en boca.
- **Citric Acid**: aporta frescura y estabilidad al vino. Una cantidad moderada se asocia con una mejor calidad sensorial, ya que contribuye al equilibrio del sabor.

## b) Realizar un análisis de regresión lineal simple entre la variable respuesta y cada variable predictora.


### Importo el Dataset

In [1]:
# importo las librerias necesarias
import pandas as pd
from scipy.stats import t
from sympy import symbols, sqrt
import numpy as np
from IPython.display import Markdown

# abro el dataset
df= pd.read_csv('winequality-red.csv', delimiter=';')

#asigno variable respuesta
display(Markdown("### Variable Respuesta"))
display(df[["quality"]])
y = df["quality"]

#asigno variables predictoras
variables_predictoras = df[['volatile acidity', 'density', 'sulphates', 'alcohol', 'citric acid']]
display(Markdown("### Variables Predictoras"))
display(variables_predictoras)

### Variable Respuesta

,quality
0,5
1,5
2,5
3,6
4,5
...,...
1594,5
1595,6
1596,6
1597,5


### Variables Predictoras

,volatile acidity,density,sulphates,alcohol,citric acid
0,0.700,0.99780,0.56,9.4,0.00
1,0.880,0.99680,0.68,9.8,0.00
2,0.760,0.99700,0.65,9.8,0.04
3,0.280,0.99800,0.58,9.8,0.56
4,0.700,0.99780,0.56,9.4,0.00
...,...,...,...,...,...
1594,0.600,0.99490,0.58,10.5,0.08
1595,0.550,0.99512,0.76,11.2,0.10
1596,0.510,0.99574,0.75,11.0,0.13
1597,0.645,0.99547,0.71,10.2,0.12


### Función de Regresion Lineal Simple

## Seleccion de X*

1. Criterio metodológico

El valor máximo de la variable independiente se encuentra dentro del rango empírico de observaciones, lo que garantiza la validez estadística del modelo y evita extrapolaciones.
Seleccionar el extremo superior permite analizar el comportamiento del modelo en el límite de los datos observados, manteniendo la coherencia con la información real disponible.

2. Criterio interpretativo

Desde un punto de vista analítico, el valor máximo de la variable predictora representa la situación de mayor intensidad o concentración del factor bajo estudio (por ejemplo, el nivel más alto de alcohol, acidez o sulfatos).
Esto permite evaluar el impacto máximo estimado de dicha característica sobre la variable dependiente (quality), ofreciendo una interpretación más tangible del efecto de la variable explicativa.

3. Consistencia entre modelos

Además, mantener el mismo criterio de selección (
𝑥∗=max(X)) en todas las regresiones asegura homogeneidad en la comparación de resultados entre las distintas variables predictoras, ya que todos los modelos se evalúan en el punto de mayor valor observado.

In [2]:
# Declaro variables generales
alpha = 0.05
n = len(y)
t_critico = t.ppf(1-alpha/2, n-2)

y_promedio = y.mean()
Syy = ((y - y_promedio) ** 2).sum()

# Funcion para calcular todos los valores que se piden a la variable predictora
def regresion_lineal(X, x, punto_x):
    
    # Declaro variables locales
    x_promedio = X.mean()
    Sxx = ((X - x_promedio) ** 2).sum()
    Sxy = ((X - x_promedio) * (y - y_promedio)).sum()
    b1 = (Sxy / Sxx).round(4)
    b0 = (y_promedio - b1 * x_promedio).round(4)
    SSR = Syy - b1 * Sxy

    # Calculo ŷ
    y_estimado = b1*x + b0

    # Calculo Varianza Estimada 
    varianza_estimada = (SSR / (n - 2)).round(4)

    # Calculo Coeficiente de Determinación
    R2 = (1 - (SSR / Syy)).round(4)

    # Calculo Coeficiente de Correlación Lineal
    r = (sqrt(R2)).round(4)

    # Calculo IC(β₀)
    e_b0 = t_critico * sqrt(varianza_estimada * (1/n + (x_promedio**2 / Sxx)))
    IC_b0_superior = (b0 + e_b0).round(4)
    IC_b0_inferior = (b0 - e_b0).round(4)

    # Calculo IC(β₁)
    e_b1 = t_critico * sqrt(varianza_estimada / Sxx)
    IC_b1_superior = (b1 + e_b1).round(4)
    IC_b1_inferior = (b1 - e_b1).round(4)

    # Calculo ICM(Y)
    e_media = t_critico * sqrt(varianza_estimada * (1/n + ((punto_x - x_promedio)**2 / Sxx))) 
    IC_media_superior = (b0 + b1*punto_x + e_media).round(4) 
    IC_media_inferior = (b0 + b1*punto_x - e_media).round(4) 

    # Calculo IP(Y)
    punto_y = (y_estimado.evalf(subs = {x: punto_x})).round(4)
    e_pred = t_critico * sqrt(varianza_estimada * (1 + 1/n + ((punto_x - x_promedio)**2 / Sxx))) 
    IC_pred_superior = (punto_y + e_pred).round(4) 
    IC_pred_inferior = (punto_y - e_pred).round(4) 


    display(Markdown(f"ŷ = {y_estimado}"))
    display(Markdown(f'Varianza estimada: {varianza_estimada}'))
    display(Markdown(f'R²: {R2}'))
    display(Markdown(f'r (correlación): {r}\n'))
    display(Markdown(f'IC(β₀): ({IC_b0_inferior}, {IC_b0_superior})'))
    display(Markdown(f'IC(β₁): ({IC_b1_inferior}, {IC_b1_superior})'))
    display(Markdown(f'ICM(Y) para x={punto_x}: ({IC_media_inferior}, {IC_media_superior})'))
    display(Markdown(f'IP(Y) para x={punto_x}, y={punto_y}: ({IC_pred_inferior}, {IC_pred_superior})\n'))

### Regresión Lineal Simple entre Volatile Acidity y Quality

In [3]:
# Calculo la regresion lineal para Volatile Acidity
x1 = symbols('x1', real=True)
volatile_acidity = variables_predictoras['volatile acidity']
x1_punto = volatile_acidity.max()
regresion_lineal(volatile_acidity,x1,x1_punto)

ŷ = 6.5657 - 1.7614*x1

Varianza estimada: 0.553

R²: 0.1525

r (correlación): 0.3905


IC(β₀): (6.4521, 6.6793)

IC(β₁): (-1.9652, -1.5576)

ICM(Y) para x=1.58: (3.5652, 4.0002)

IP(Y) para x=1.58, y=3.7827: (2.3080, 5.2574)


### Regresión Lineal Simple entre Density y Quality

In [4]:
# Calculo la regresion lineal para Density
x2 = symbols('x2', real=True)
density = variables_predictoras['density']
x2_punto = density.max()
regresion_lineal(density,x2,x2_punto)

ŷ = 80.2385 - 74.846*x2

Varianza estimada: 0.6326

R²: 0.0306

r (correlación): 0.1749


IC(β₀): (59.6279, 100.8491)

IC(β₁): (-95.5238, -54.1682)

ICM(Y) para x=1.00369: (4.9675, 5.2651)

IP(Y) para x=1.00369, y=5.1163: (3.5492, 6.6834)


### Regresión Lineal Simple entre Sulphates y Quality

In [5]:
# Calculo la regresion lineal para Sulphates
x3 = symbols('x3', real=True)
sulphates = variables_predictoras['sulphates']
x3_punto = sulphates.max()
regresion_lineal(sulphates,x3,x3_punto)

ŷ = 1.1977*x3 + 4.8478

Varianza estimada: 0.6113

R²: 0.0632

r (correlación): 0.2514


IC(β₀): (4.6940, 5.0016)

IC(β₁): (0.9714, 1.4240)

ICM(Y) para x=2.0: (6.9371, 7.5493)

IP(Y) para x=2.0, y=7.2432: (5.6794, 8.8070)


### Regresión Lineal Simple entre Alcohol y Quality

In [6]:
# Calculo la regresion lineal para Alcohol
x4 = symbols('x4', real=True)
alcohol = variables_predictoras['alcohol']
x4_punto = alcohol.max()
regresion_lineal(alcohol,x4,x4_punto)

ŷ = 0.3608*x4 + 1.8754

Varianza estimada: 0.5046

R²: 0.2267

r (correlación): 0.4761


IC(β₀): (1.5327, 2.2181)

IC(β₁): (0.3281, 0.3935)

ICM(Y) para x=14.9: (7.1008, 7.4018)

IP(Y) para x=14.9, y=7.2513: (5.8499, 8.6527)


### Regresión Lineal Simple entre Citric Acid y Quality

In [7]:
# Calculo la regresion lineal para Citric Acid
x5 = symbols('x5', real=True)
citric_acid = variables_predictoras['citric acid']
x5_punto = citric_acid.max()
regresion_lineal(citric_acid,x5,x5_punto)

ŷ = 0.9385*x5 + 5.3817

Varianza estimada: 0.6191

R²: 0.0512

r (correlación): 0.2263


IC(β₀): (5.3156, 5.4478)

IC(β₁): (0.7403, 1.1367)

ICM(Y) para x=1.0: (6.1706, 6.4698)

IP(Y) para x=1.0, y=6.3202: (4.7696, 7.8708)


### Tabla resumen

| $Y$          | $\hat{y}$      | $\hat{σ^2}$       | $R^2$     | $r$      | IC($β₀$)             | IC($β₁$)             | ICM($Y$)                | IP($Y$)                 |
|-------------------|---------|-----------|--------|--------|------------------|------------------|---------------------|---------------------|
| Volatile Acidity ($x_1$)                | -1.7614*$x_1$ + 6.5657  | 0.553    | 0.1525 | 0.3905 | (6.4521, 6.6793) | (-1.9652, -1.5576) | (3.5651, 4.0001)    | (2.3078, 5.2574)    |
| Density ($x_2$)                | -74.846*$x_2$ + 80.2385  | 0.6326    | 0.0305 | 0.1749 | (59.6279, 100.8491) | (-95.5238, -54.1682) | (4.9675, 5.2651)    | (3.5491, 6.6834)    |
| Sulphates ($x_3$)                | 1.1977*$x_3$ + 4.8477  | 0.6113    | 0.0632 | 0.2513 | (4.6940, 5.0016) | (0.9713, 1.424) | (6.937, 7.5492)    | (5.6793, 8.807)    |
| Alcohol ($x_4$)                | 0.3608*$x_4$ + 1.8749|  0.5046| 0.2267| 0.4761| (1.5327,  2.2181) | (0.3281, 0.3935) | (7.1009, 7.4020)    | (5.8500, 8.6529)    |
| Citric Acid ($x_5$)                | 0.9384*$x_5$ + 5.3817  | 0.6191    | 0.0512| 0.2263| (5.3156, 5.4478) | (0.7402, 1.1366) | (6.1706, 6.4697)    | (4.7695, 7.8707)    |

## c) Seleccionar la variable predictora que mejor responde a la variable respuesta y comentar los resultados obtenidos en el cuadro sobre la misma.

Para saber que variable responde mejor, hay que analizar el **coeficiente de correlación lineal** de cada variable predictora. En base a la tabla, tenemos que:

$|r_4| > |r_1|, |r_2|, |r_3|, |r_5|$

Por lo que la variable **Alcohol** ($x_4$) es la que mejor responde a **Quality**

# Regresión Lineal Múltiple

## d) Estimar la ecuación de regresión usando el método de descenso del gradiente.

El método de descenso del gradiente se basa en la optimización iterativa de los parámetros del modelo.
 A partir de un valor inicial de los coeficientes, el algoritmo ajusta los parámetros en la dirección opuesta al gradiente de la función de costo (error cuadrático medio), con el objetivo de minimizar el error global.
Este enfoque no depende de la resolución analítica de un sistema de ecuaciones, sino que ajusta los parámetros gradualmente hasta alcanzar la convergencia. Por eso es especialmente útil cuando se trabaja con grandes volúmenes de datos o cuando el cálculo matricial exacto resulta costoso computacionalmente.

### Variables Generales

In [ ]:
normalized = (variables_predictoras - variables_predictoras.mean()) / variables_predictoras.std()

### Descenso de Gradiente

In [54]:
# REGRESION LINEAL MULTIPLE

# Agregar columna de 1s para el intercepto
X_b = np.c_[np.ones((normalized.shape[0], 1)), normalized]

# Inicialización
m, n = X_b.shape
beta_grad = np.zeros(n)
alpha = 0.01
epochs = 1000
min_err = 1e-6
# Descenso del gradiente
for epoch in range(epochs):
    y_pred = X_b.dot(beta_grad)
    error = y_pred - y
    grad = (1/m) * X_b.T.dot(error)
    beta_grad -= alpha * grad

    # Verificar si el error es menor que el mínimo permitido
    if np.linalg.norm(grad, ord=1) < min_err:
        print(f"Convergió en la época {epoch}")
        break

print("Coeficientes estimados:", beta_grad)
y_predicted = beta_grad[0].round(4) + beta_grad[1].round(4)*x1 + beta_grad[2].round(4)*x2 + beta_grad[3].round(4)*x3 + beta_grad[4].round(4)*x4 + beta_grad[5].round(4)*x5
print("ŷ =", y_predicted)
y_pred_grad = X_b.dot(beta_grad)

Coeficientes estimados: [ 5.6357792  -0.22613062  0.0178916   0.11597664  0.33703191 -0.0190865 ]
ŷ = -0.2261*x1 + 0.0179*x2 + 0.116*x3 + 0.337*x4 - 0.0191*x5 + 5.6358


## e) Estimar la ecuación de regresión usando el método de mínimos cuadrados. Comentar los resultados obtenidos por ambos métodos.

El método de mínimos cuadrados calcula directamente los coeficientes que minimizan la suma de los errores al cuadrado, resolviendo el sistema normal de ecuaciones de forma exacta y determinística.
 A diferencia del descenso del gradiente, este procedimiento no depende de una tasa de aprendizaje ni de iteraciones sucesivas, por lo que ofrece una solución única y estable cuando las condiciones del modelo son adecuadas.


### Mínimos Cuadrados

In [53]:
# Cálculo de los coeficientes usando minimos cuadrados

# Añadimos columna de 1s para el intercepto
Z_b = np.c_[np.ones(normalized.shape[0]), normalized]
# Coeficientes de regresión
beta_cuad = np.linalg.inv(Z_b.T.dot(Z_b)).dot(Z_b.T).dot(y)
print("Coeficientes estimados:", beta_cuad)
y_predicted = beta_cuad[0].round(4) + beta_cuad[1].round(4)*x1 + beta_cuad[2].round(4)*x2 + beta_cuad[3].round(4)*x3 + beta_cuad[4].round(4)*x4 + beta_cuad[5].round(4)*x5
print("ŷ =", y_predicted)
y_pred_cuad = Z_b.dot(beta_cuad)

Coeficientes estimados: [ 5.63602251 -0.23304422  0.02850916  0.11520628  0.34407703 -0.03019699]
ŷ = -0.233*x1 + 0.0285*x2 + 0.1152*x3 + 0.3441*x4 - 0.0302*x5 + 5.636


### Conclusión
Comparando ambos métodos, se observa que los coeficientes estimados son muy similares en valor y signo, lo que indica una convergencia exitosa del descenso del gradiente hacia la misma solución óptima.
 Las pequeñas diferencias decimales se explican por los criterios de parada y tolerancia del algoritmo iterativo, pero no alteran la interpretación del modelo.

En síntesis, ambos métodos describen una relación lineal consistente entre la calidad del vino y las variables predictoras seleccionadas. Las mayores contribuciones positivas corresponden al alcohol y a los sulfitos, mientras que la acidez volátil presenta un efecto negativo, confirmando los patrones observados en los modelos univariados y en la literatura enológica.

## f) ¿La adición de más variables predictoras mejoró la estimación en comparación con la obtenida en el inciso c)? Explique.

Para ver si la adiccion de variables mejora la estimacion hay que calcular el coeficiente de correlacion multiple (rₐ), y compararlo con el coeficiente de correlacion (r) de la variable alcohol

### Cálculo de $r_a$

In [46]:
SSr = ((y - y_pred_cuad)**2).sum()
SSt = ((y - y.mean())**2).sum()
R2 = 1 - SSr/SSt
print("R²:", R2)
k = 5
Ra2 = 1 - (1-R2)*(len(y)-k)/(len(y) - k - 1)
print("Rₐ² ajustado:", Ra2)
ra = sqrt(Ra2)
print("rₐ (correlación):", ra)

R²: 0.3368070479860976
Rₐ² ajustado: 0.3363907310042936
rₐ (correlación): 0.579992009431418


1. Resultado de multiples variables predictoras:
    - rₐ (correlación): 0.579992009431418

2. Resultado de una unica variable predictora:
    - r (correlación): 0.4761

3. Conclusion:
    - Como |rₐ| > |r| se demuesta que la correlacion al utilizar una unica variable predictora (alcohol) es mas debil que la correlacion utilizando mas variables predictoras (volatile acidity, density, sulphates, alcohol, citric acid). Por lo tanto se demuestra que la adicion de mas variables predictoras mejoro la estimacion en comparacion con la estimacion del inciso c).